# 02. Generating Multiple Samples using MS1 Controller

In this notebook, we demonstrate how ViMMS can be used to generate multiple samples (sets of chemicals) that are biological and technical replicates. The MS1 controller is then used to produce mass spectral data in form of .mzML files for the multiple samples.

In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import sys
from collections import defaultdict
import os

In [4]:
sys.path.append('..')

In [5]:
from vimms.Chemicals import ChemicalCreator, MultiSampleCreator
from vimms.MassSpec import IndependentMassSpectrometer
from vimms.Controller import SimpleMs1Controller
from vimms.Common import *

Load previously trained KDEs in `PeakSampler` and the list of extracted metabolites, created in **01. Download Data.ipynb**.

In [6]:
base_dir = os.path.abspath('example_data')
ps = load_obj(os.path.join(base_dir, 'peak_sampler_mz_rt_int_19_beers_fullscan.p'))
hmdb = load_obj(os.path.join(base_dir, 'hmdb_compounds.p'))

Set ViMMS logging level

In [7]:
set_log_level_warning()
# set_log_level_info()
# set_log_level_debug()

## Create Initial Chemical

Define an output folder containing our results

In [8]:
out_dir = os.path.join(base_dir, 'MS1_demo')

Here we generate multiple chemical objects that will be used across samples. The chemical objects are generated by sampling from metabolites in the HMDB database.

In [9]:
# the list of ROI sources created in the previous notebook '01. Download Data.ipynb'
ROI_Sources = [os.path.join(base_dir,'DsDA\\DsDA_Beer\\beer_t10_simulator_files\\')]

# minimum MS1 intensity of chemicals
min_ms1_intensity = 1.75E5

# m/z and RT range of chemicals
rt_range = [(400, 800)]
mz_range = [(100, 400)]

# the number of chemicals in the sample
n_chems = 1000

# maximum MS level (we do not generate fragmentation peaks when this value is 1)
ms_level = 1

# for this experiment, we restrict the sampled chromatograms to be within 20 - 40s in length
# so they are not too big and too small
roi_rt_range = [20, 40]

In [10]:
chems = ChemicalCreator(ps, ROI_Sources, hmdb)
dataset = chems.sample(mz_range, rt_range, min_ms1_intensity, n_chems, 1, 
                       fixed_mz=False, roi_rt_range=roi_rt_range)
save_obj(dataset, os.path.join(out_dir, 'BaseDataset\\dataset.p'))

Created C:\Users\joewa\Work\git\vimms\examples\example_data\MS1_demo\BaseDataset
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\MS1_demo\BaseDataset\dataset.p


In [11]:
for chem in dataset[0:10]:
    print(chem)

KnownChemical - 'C12H22' rt=565.56 max_intensity=69194903.29
KnownChemical - 'C5H12O11P2' rt=404.16 max_intensity=6119934.20
KnownChemical - 'C16H24N2' rt=575.36 max_intensity=1266660.94
KnownChemical - 'C18H38O' rt=649.90 max_intensity=915285.24
KnownChemical - 'C6H11N3' rt=551.89 max_intensity=1149830.51
KnownChemical - 'Cl3N' rt=753.61 max_intensity=1429321.84
KnownChemical - 'C8H13NO' rt=557.24 max_intensity=374678.76
KnownChemical - 'C3H4Cl2' rt=548.22 max_intensity=257774.74
KnownChemical - 'C13H12O7S' rt=510.58 max_intensity=395485.84
KnownChemical - 'C6H5BrO' rt=576.05 max_intensity=1496285.96


## Create Multiple Samples

The next section allows us to define classes of biological replicates, each having multiple technical replicates. 

Below we create two biological classes ('class0', 'class1'), each having 10 technical replicates with some noise on the chemical's intensity.

In [12]:
n_samples = [10, 10] # number of files per class
classes = ["class%d" % i for i in range(len(n_samples))] # creates default list of classes
intensity_noise_sd = [1000] # noise on max intensity

In [13]:
classes

['class0', 'class1']

Add intensity changes between different classes

In [14]:
change_probabilities = [0 for i in range(len(n_samples))] # probability of intensity changes between different classes
change_differences_means = [0 for i in range(len(n_samples))] # mean of those intensity changes
change_differences_sds = [0 for i in range(len(n_samples))] # SD of those intensity changes

Add experimental variables (examples in comments)

In [15]:
experimental_classes = None # [["male","female"],["Positive","Negative","Unknown"]]
experimental_probabilitities = None # [[0.5,0.5],[0.33,0.33,0.34]]
experimental_sds = None # [[250],[250]]

Dropout chemicals in different classes

In [16]:
# drop-out chemicals by their probabilities
# dropout_probability = 0.2
# dropout_probabilities = [dropout_probability for i in range(len(n_samples))]

# drop-out chemicals by an absolute number
dropout_probabilities = None
dropout_numbers = 2 # number of chemicals dropped out in each class

Generate multiple samples

In [17]:
save_location = os.path.join(out_dir, 'ChemicalFiles')

In [18]:
multiple_samples = MultiSampleCreator(dataset, n_samples, classes, intensity_noise_sd, 
                                      change_probabilities, change_differences_means, change_differences_sds, dropout_probabilities, dropout_numbers,
                                     experimental_classes, experimental_probabilitities, experimental_sds, save_location=save_location)

Created C:\Users\joewa\Work\git\vimms\examples\example_data\MS1_demo\ChemicalFiles
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\MS1_demo\ChemicalFiles\sample_0.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\MS1_demo\ChemicalFiles\sample_1.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\MS1_demo\ChemicalFiles\sample_2.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\MS1_demo\ChemicalFiles\sample_3.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\MS1_demo\ChemicalFiles\sample_4.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\MS1_demo\ChemicalFiles\sample_5.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\MS1_demo\ChemicalFiles\sample_6.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\MS1_demo\ChemicalFiles\sample_7.p
Saving <class 'list'> to C:\U

In [19]:
total_samples = np.sum(multiple_samples.n_samples)
total_samples

20

We can also print the chemicals that are missing (removed by drop-out) in each class.

In [20]:
save_obj(multiple_samples.missing_chemicals, os.path.join(out_dir, 'MissingChemicals\\missing_chemicals.p'))
multiple_samples.missing_chemicals

Created C:\Users\joewa\Work\git\vimms\examples\example_data\MS1_demo\MissingChemicals
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\MS1_demo\MissingChemicals\missing_chemicals.p


[[KnownChemical - 'C6H6Cl2N2O4S2' rt=441.72 max_intensity=308314.42,
  KnownChemical - 'C10H19N3O4' rt=480.64 max_intensity=352228.27],
 [KnownChemical - 'C9H15NO2' rt=603.27 max_intensity=898049.53,
  KnownChemical - 'C6H5ClO5' rt=438.56 max_intensity=1321042.52]]

## Run MS1 controller on the samples and generate .mzML files

We can now take the multiple samples created above and generate mass spectral data (.mzML files) using the MS1 controller in ViMMS.

In [21]:
min_rt = rt_range[0][0]
max_rt = rt_range[0][1]
controllers = defaultdict(list)
controller_to_mzml = {}

mzml_dir = os.path.join(out_dir, 'mzMLFiles')
num_classes = len(n_samples)
sample_idx = 0
for j in range(num_classes): # loop over classes
    num_samples = n_samples[j]
    for i in range(num_samples): # loop over samples for each class
        
        # load the sample
        fname = os.path.join(save_location, 'sample_%d.p' % sample_idx) 
        sample = load_obj(fname)
        sample_idx += 1
        
        # define output .mzML filename
        out_file = 'number_%d_class_%d.mzML' % (i, j)
        out_path = os.path.join(mzml_dir, out_file)
        print('Generating %s' % out_path)

        # run it through the MS1 controller        
        mass_spec = IndependentMassSpectrometer(POSITIVE, sample, density=ps.density_estimator)
        controller = SimpleMs1Controller(mass_spec)
        controller.run(min_rt,max_rt)
        controller.write_mzML('my_analysis', out_path)

        # save the resulting controller
        controllers[j].append(controller)
        controller_to_mzml[controller] = (j, out_file, )

Generating C:\Users\joewa\Work\git\vimms\examples\example_data\MS1_demo\mzMLFiles\number_0_class_0.mzML


400.89137000000073it [00:05, 79.45it/s]                                                                                           


Created C:\Users\joewa\Work\git\vimms\examples\example_data\MS1_demo\mzMLFiles
Generating C:\Users\joewa\Work\git\vimms\examples\example_data\MS1_demo\mzMLFiles\number_1_class_0.mzML


401.07822699999906it [00:04, 81.28it/s]                                                                                           


Generating C:\Users\joewa\Work\git\vimms\examples\example_data\MS1_demo\mzMLFiles\number_2_class_0.mzML


...

400.0698999999996it [00:05, 104.25it/s]                                                                                           


Generating C:\Users\joewa\Work\git\vimms\examples\example_data\MS1_demo\mzMLFiles\number_8_class_1.mzML


400.9648799999983it [00:04, 84.71it/s]                                                                                            


Generating C:\Users\joewa\Work\git\vimms\examples\example_data\MS1_demo\mzMLFiles\number_9_class_1.mzML


400.58418000000063it [00:05, 79.32it/s]                                                                                           


## Print out the missing peaks

The controller object contains all the information about the state of the mass spectrometry process over time. Below we demonstrate this by generating a report of peaks corresponding to a chemical that are present in one class but is missing from the other class. This can be useful in the benchmark evaluation of peak picking or alignment algorithms.

In [22]:
def get_chem_to_peaks(controller):
    chem_to_peaks = defaultdict(list)
    frag_events = controller.mass_spec.fragmentation_events
    for frag_event in frag_events:
        chem = frag_event.chem
        peaks = frag_event.peaks
        chem_to_peaks[chem].extend(peaks)
    return chem_to_peaks

In [23]:
for controller, (current_class, mzml_filename) in controller_to_mzml.items():
    controller_peaks = get_chem_to_peaks(controller)
    basename = os.path.basename(mzml_filename)
    front, back = os.path.splitext(mzml_filename)
    outfile = front + '.csv'

    missing_peaks = []            
    for other_class in range(num_classes):
        if current_class == other_class:
            continue

        # get the peaks that are present in current_class but missing in other_class
        missing_chems = multiple_samples.missing_chemicals[other_class]
        for chem in missing_chems:
            peaks = controller_peaks[chem]
            for peak in peaks:
                row = (chem.formula.formula_string, current_class, other_class, peak.mz, peak.rt, peak.intensity)
                missing_peaks.append(row)
    
    # convert to dataframe
    columns = ['formula', 'present_in', 'missing_in', 'mz', 'RT', 'intensity']
    missing_df = pd.DataFrame(missing_peaks, columns=columns)
    missing_df.to_csv(os.path.join(out_dir, 'MissingChemicals', os.path.basename(outfile)))

In [24]:
missing_df.head(20)

,formula,present_in,missing_in,mz,RT,intensity
0,C6H6Cl2N2O4S2,1,0,304.921890,419.9115,224417.245511
1,C6H6Cl2N2O4S2,1,0,345.948437,419.9115,45298.978018
2,C6H6Cl2N2O4S2,1,0,608.836493,419.9115,28678.697827
3,C6H6Cl2N2O4S2,1,0,305.925245,419.9115,14976.277253
4,C6H6Cl2N2O4S2,1,0,304.921958,421.1615,271726.183831
5,C6H6Cl2N2O4S2,1,0,345.948505,421.1615,54848.362479
6,C6H6Cl2N2O4S2,1,0,608.836561,421.1615,34724.395178
7,C6H6Cl2N2O4S2,1,0,305.925313,421.1615,18133.395483
8,C6H6Cl2N2O4S2,1,0,304.921941,422.4845,347040.611090
9,C6H6Cl2N2O4S2,1,0,345.948488,422.4845,70050.699434
